In [1]:
import requests
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import ssl
import pandas as pd 
import spacy
from spellchecker import SpellChecker
from translate import Translator

In [165]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

nltk.download('punkt')
nltk.download('stopwords')

In [2]:
#  разбиваю на слова
file_path = '/Users/veronikaplisova/trash/visual_studio/en_project/files/The Ballad of Songbirds and Snakes - Suzanne Collins.txt'
# file_path = '/Users/veronikaplisova/trash/visual_studio/en_project/files/text_sample.txt'

punctuation_marks = ['’','!', ',', '(', ')', ':', ';', '-', '?', '.', '“', '"', '—', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
content_separated = ''.join([' ' + char + ' ' if char in punctuation_marks else char for char in content])


content_limit = content_separated#[:10000]

words = word_tokenize(content_limit.lower())

only_words = []
for word in words:
    if not any(char in punctuation_marks for char in word):
        only_words.append(word)

unique_words = only_words #list(set(only_words))
    
len(unique_words)

163940

In [ ]:
# привожу к инфинитивной форме
nlp = spacy.load("en_core_web_sm")

lemmas = []
for word in unique_words:
    # print(word)
    doc = nlp(word)
    for token in doc:
        # print(token)
        text = token.text
        lemma = token.lemma_
        pos = token.pos_
        tag = token.tag_
        # print(f"Token: {text}, Lemma: {lemma}, POS: {pos}, Tag: {tag}")
        lemmas.append(lemma)

unique_lemmas = lemmas #list(set(lemmas))

len(unique_lemmas)

In [ ]:
# убираем бессмысленные слова
stop_words = stopwords.words("english")
filtered_lemmas = []
for token in unique_lemmas:
    if token not in stop_words:
        filtered_lemmas.append(token)

len(filtered_lemmas)

7434

In [169]:
lemmas_df = pd.DataFrame(filtered_lemmas, columns= ['word'])
len(lemmas_df)

7434

In [170]:
# убираем слова, которых не сущетсвует
spell = SpellChecker()

lemmas_df['correct_word'] = lemmas_df['word'].apply(spell.correction)

In [171]:
# чистим слова, где 2 или меньше уникальных букв
correct_words_df = lemmas_df[ ~lemmas_df['correct_word'].isna() ]

correct_words_df['distinct_letter_count'] = correct_words_df['correct_word'].apply(lambda x: len(set(x)))

correct_words_df = correct_words_df[correct_words_df['distinct_letter_count'] > 2]

correct_words_df = correct_words_df[['correct_word']]

len(correct_words_df)

/var/folders/tg/z11g391j2wd3_2qz7ypb1tgm0000gn/T/ipykernel_11524/1769115894.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correct_words_df['distinct_letter_count'] = correct_words_df['correct_word'].apply(lambda x: len(set(x)))


7268

In [172]:
correct_words_df

,correct_word
0,ancient
1,topple
2,ballad
3,sidestep
4,sure
...,...
7429,polite
7430,electricity
7431,breach
7432,brow


In [173]:
correct_words_df.drop_duplicates()
correct_words_df.columns = ['word']
correct_words_df

,word
0,ancient
1,topple
2,ballad
3,sidestep
4,sure
...,...
7429,polite
7430,electricity
7431,breach
7432,brow


In [174]:
correct_words_df.to_csv('/Users/veronikaplisova/trash/visual_studio/en_project/files/en_words.csv', index=False)

In [175]:
def translate_to_russian(word):
    translator = Translator(to_lang="ru")
    translation = translator.translate(word)
    return translation

In [176]:
correct_words_df['word_ru'] = correct_words_df['word'].apply(translate_to_russian)

In [177]:
correct_words_df

,word,word_ru
0,ancient,Древний
1,topple,Чебурашка
2,ballad,баллады
3,sidestep,sidestep
4,sure,конечно
...,...,...
7429,polite,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...
7430,electricity,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...
7431,breach,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...
7432,brow,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...


In [178]:
correct_words_df.to_csv('/Users/veronikaplisova/trash/visual_studio/en_project/files/en_ru_words.csv', index=False)